### Настройка Airflow

Для начала вам необходимо выполнить ряд команд чтобы настроить окружение для дальнейшей работы, это позволит первое время не заниматься настройкой среды исполнения, а сразу начать писать код и работать с Airflow.

In [ ]:
# Установка Airflow
!pip install apache-airflow==2.1.4

# Инициализация базы данных
!airflow db init

In [ ]:
# Создадим необходимые папки
!mkdir /root/airflow/dags
!touch /root/airflow/dags/dag.py

In [ ]:
# Включим веб-сервер
!airflow webserver -p 18273 -D

In [ ]:
# Создадим пользователя Airflow
!airflow users create \
          --username admin \
          --firstname admin \
          --lastname admin \
          --role Admin \
          --email admin@example.org \
          -p 12345

Поместите в dag.py следующий код.

```python
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago
from airflow.operators.dummy_operator import DummyOperator

dag = DAG('dag',schedule_interval=timedelta(days=1), start_date=days_ago(1))
t1 = DummyOperator(task_id='task_1', dag=dag)
t2 = DummyOperator(task_id='task_2',dag=dag)
t3 = DummyOperator(task_id='task_3',dag=dag)
t4 = DummyOperator(task_id='task_4',dag=dag)
t5 = DummyOperator(task_id='task_5',dag=dag)
t6 = DummyOperator(task_id='task_6',dag=dag)
t7 = DummyOperator(task_id='task_7',dag=dag)

[t1, t2]>>t5
t3>>t6
[t5,t6] >>  t7
t4
```

In [ ]:
# Запуск шедулера
!airflow scheduler -D

In [ ]:
# Последующие команды не имеют отношения к Airflow
# Они нужни только для корректной работы веб морды
# в среде Google Colab

!pip install pyngrok
!ngrok authtoken <YOUR TOKEN> # найти его можно https://dashboard.ngrok.com/get-started/setup 

# Эта команда просто отображет веб морду на другой адрес
# Его вы можете найти https://dashboard.ngrok.com/cloud-edge/status
# При каждом отключении ссылка будет меняться
!nohup ngrok http -log=stdout 18273 > /dev/null &

После запуска команды выше, перейдите по адресу в ngrok и подождите  пока появится DAG с именем dag

### Практика

**Весь код необходимо вставлять в dag.py**

### Пример Хука и Оператора

Приведу пример самого простого хука и оператора.

**Задание**

Результат данного кода отправит в Xcom некоторое число, оно и будет ответом на вопрос.

```python
from airflow.hooks.base import BaseHook
from airflow.models import BaseOperator
from airflow import DAG
from datetime import timedelta
from airflow.utils.dates import days_ago

import random
 

# Кастомный хук 
class CustomHook(BaseHook):
 
    # Метод который генерирует случайное число
    def random_number(self):
      random.seed(10, version=2)
      return random.randint(0, 10)

# Кастомный оператор
class CustomOperator(BaseOperator):
    def __init__(self,**kwargs,):
        super().__init__(**kwargs)     
        self.hook = None # Сюда мы будем передавать объект класса CustomHook

    # Метод отправляет в Xcom некотрое значение
    def execute(self, context):
      
      self.hook = CustomHook()
      return self.hook.random_number()

dag = DAG('dag', schedule_interval=timedelta(days=1), start_date=days_ago(1))

t1 = CustomOperator(task_id='task_1', dag=dag)
```

